In [ ]:
import h5py
import numpy as np
fh5 = h5py.File('d:/h5/fmap_cf.h5','r')
img = fh5['map']
img = np.array(img)
lab = fh5['lab']
lab = np.array(lab)

In [ ]:
img = img.reshape(len(img), 33, 33, 1)

In [ ]:
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(lab)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(img,train_labels,test_size=0.25,random_state=1)

In [ ]:
from keras import models
from keras import layers
from keras.models import Sequential,Input
from keras import optimizers
import keras
from keras.layers import Conv2D,MaxPooling2D,Dropout,BatchNormalization,Flatten,Dense,Lambda,Multiply,AveragePooling2D,Activation
from keras.models import Model

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=10),
    keras.callbacks.ModelCheckpoint(filepath='e:/data/rmap.h5',monitor='val_loss',save_best_only=True)   ##保存最佳模型
    ]

In [ ]:
def Conv2d_BN(x, nb_filter,kernel_size, padding='valid',strides=(1,1),data_format='channels_last',dilation_rate=(1,1),
              activation='relu',name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,data_format='channels_last',activation='relu',
               dilation_rate=dilation_rate,name=conv_name)(x)
    x = BatchNormalization(name=bn_name)(x)
    
    return x

In [ ]:
def InceptionB(x,nb_filter):
    branch_1x1_t = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_1x1_s = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_1x1_s = Activation('sigmoid')(branch_1x1_s)
    branch_1x1_c = keras.layers.Multiply()([branch_1x1_t,branch_1x1_s])
    
    branch_3x3_t = Conv2d_BN(x,nb_filter,(3,3), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_3x3_s = Conv2d_BN(x,nb_filter,(3,3), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_3x3_s = Activation('sigmoid')(branch_3x3_s)
    branch_3x3_c = keras.layers.Multiply()([branch_3x3_t,branch_3x3_s])
       
    branch_5x5_t = Conv2d_BN(x,nb_filter,(5,5), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_5x5_s = Conv2d_BN(x,nb_filter,(5,5), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_5x5_s = Activation('sigmoid')(branch_5x5_s)
    branch_5x5_c = keras.layers.Multiply()([branch_5x5_t,branch_5x5_s])
    

    x = keras.layers.Concatenate(axis=3)([branch_1x1_c,branch_3x3_c,branch_5x5_c]) #

    return x

In [ ]:
def InceptionC(x,nb_filter):
    branch_1x1_t = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_1x1_s = Conv2d_BN(x,nb_filter,(1,1), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_1x1_s = Activation('sigmoid')(branch_1x1_s)
    branch_1x1_c = keras.layers.Multiply()([branch_1x1_t,branch_1x1_s])
    
    branch_3x3_t = Conv2d_BN(x,nb_filter,(3,3), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_3x3_s = Conv2d_BN(x,nb_filter,(3,3), padding='same',strides=(1,1),data_format='channels_last',activation='relu',name=None)
    branch_3x3_s = Activation('sigmoid')(branch_3x3_s)
    branch_3x3_c = keras.layers.Multiply()([branch_3x3_t,branch_3x3_s])
    

    x = keras.layers.Concatenate(axis=3)([branch_1x1_c,branch_3x3_c]) #branch_1x1_c,

    return x

In [ ]:
def TestNet(input_shape=None):
    inpt = Input(shape=input_shape)

    x = Conv2d_BN(inpt,32,(3,3),strides=(1,1),padding='same')
    x = InceptionB(x,64)
    x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
    x = InceptionB(x,128)
    x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
    x = InceptionC(x,256)
    x = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
    x = InceptionC(x,256)
    x = AveragePooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x)
    x = Dropout(0.3)(x)
    x = Flatten()(x)
    x = Dense(128,activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(28,activation='softmax')(x)
    model = Model(inpt,x,name='inception')
    
 
    return model

In [ ]:
model = TestNet(input_shape=(33,33,1))
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model     ##模型结构图
plot_model(model, 'd:\\h5\\model_3d.png')

In [ ]:
plot_model(model, 'd:\\h5\\model_3d.png',show_shapes=True)

In [ ]:
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.adam_v2.Adam(lr=1e-5),#
              metrics=['acc']
             )

In [ ]:
his = model.fit(X_train,y_train,epochs=1,batch_size=64)

In [ ]:
test_loss,test_acc= model.evaluate(X_test,y_test)
print('test_acc:',test_acc)

In [ ]:
predict = model.predict(X_test)
predict=np.argmax(predict,axis=1)

In [ ]:
y_test2=np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import classification_report
print('查准率、查全率、F1值：')
print(classification_report(y_test2,predict,target_names=None))

In [ ]:
model.save_weights('d:/h5/fmap_cf_weights.h5')